In [7]:
import polars as pl
from hashlib import sha256
import os
import json
from urllib.parse import urlparse, parse_qs
import networkx as nx
import matplotlib.pyplot as plt
pl.Config.set_fmt_str_lengths(400)
# !pip install pyarrow

polars.config.Config

In [8]:
DATA_PATH = os.getenv("DATA_PATH", "")

In [9]:
links_lazy_df = pl.scan_parquet(f"{DATA_PATH}/farcaster-links-0-1730134800.parquet")
bot_or_not_df = pl.scan_parquet(f"{DATA_PATH}/bot_or_not_with_info.parquet")

In [10]:
bot_or_not_df.head(1).collect()

created_at,updated_at,custody_address,registered_at,fid,verifications_count,following_count,followers_count,recasts_count,likes_count,total_casts_count,reply_casts_count,casts_count,bot,text
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,u32,u32,u32,u32,u32,u32,u32,u32,bool,str
2024-04-04 00:34:26.279484,2024-06-07 08:11:42.199,"b""\xcb\x1b\xbc\x1e\xbd;\xb6\x8dz\xe9\xf1\xc2\xf0\x1c\x0c\xc8\x8a\xf3\xd4\xef""",2024-04-04 00:34:11 UTC,446097,1,1024,902,155,869,339,308,31,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic comments"""


In [11]:
# bot_or_not_df.select(["fid", "bot"]).sink_parquet(f"{DATA_PATH}/bot_or_not_without_info.parquet")

In [12]:
bot_or_not_df

In [13]:
# links_lazy_df.filter(pl.col('deleted_at').is_null()).head(5).collect()

In [14]:
bot_or_not_fids = bot_or_not_df.select("fid")

In [15]:
links_filtered_by_fid = links_lazy_df.join(bot_or_not_fids, on="fid", how="inner")
links_between_bots = links_filtered_by_fid.join(bot_or_not_fids, left_on="target_fid", right_on="fid", how="inner")

In [16]:
links_between_bots_df = links_between_bots.collect()

In [17]:
links_between_bots_df

fid,target_fid,hash,timestamp,created_at,updated_at,deleted_at,type,display_timestamp,id
i64,i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64
2,23,"b""\xf6\xb3y\x03f\x0b\x0f\x15""R\xf32\xf1orQ\x16\xc1[\xcb""",2021-07-22 21:38:55,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,22
2,18,"b""\xb9\x18\xc8\xf8\xb1_\xf4SU\x9b\x0f\x18\xce\x99D\xc5T\x12\x8a\x7f""",2021-07-22 21:38:59,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,23
2,20,"b""v\xb8\xaa\x96\x1c\x94\x8d\xaf\xc0c\xbf\xb8N\x88\x82\x87\xa6Q^q""",2021-07-22 21:39:03,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,25
2,8,"b""`e\xed\xd6\xb7\x9b\xf4H\x9d\x8fU\xcb\x98\x07\xac\x15kD\x95|""",2021-07-22 21:39:04,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,26
2,12,"b""4{*K\x02l\x05\x08\xe3\x88\x84\xc5\xc2\xfb\xb6m\x9c=i\xdd""",2021-07-22 21:39:06,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,28
…,…,…,…,…,…,…,…,…,…
467312,389808,"b""\x07\xf9`&\xd6\x0foq\xce\x7f\x1d\xe7\xa9\x86B\x9f\xba#+t""",2024-09-12 12:08:20,2024-09-12 12:08:20.457983,2024-10-03 16:56:28.567,2024-10-03 16:56:29,"""follow""",null,11873708645
368382,387244,"b""\xef\xe0\x09\x8f\xb0\x02b9\x06\xab\x15\xa6\x8di\x81v\x9ezR\xbd""",2024-10-03 16:56:46,2024-09-17 16:04:19.132916,2024-10-03 16:56:46.044,null,"""follow""",null,11874838125
379818,294370,"b""\x94\x90\xfc\x96\xf0\xf6\xbe\x7fME-r\x9e\xc2\xd0\xcd\xf2J\x1bH""",2024-10-03 16:56:52,2024-10-03 16:56:52.402420,2024-10-03 16:56:52.402420,null,"""follow""",null,11878799175


In [18]:
links_df = links_between_bots_df.filter(pl.col('fid') != pl.col('target_fid'))

In [19]:
# bots_fids.collect()

In [20]:
following_count_df = (
    links_df
    .filter((pl.col("type") == "follow") & (pl.col("deleted_at").is_null()))
    .group_by("fid")
    .agg(pl.count("id").alias("following_count"))
)

In [21]:
followers_count_df = (
    links_df
    .filter((pl.col("type") == "follow") & (pl.col("deleted_at").is_null()))
    .group_by("target_fid")
    .agg(pl.count("id").alias("followers_count"))
    .rename({"target_fid": "fid"})
)

In [22]:
bots_df = bot_or_not_df.collect()
bots_df = bots_df.drop(["followers_count", "following_count"])
result_df = bots_df.join(following_count_df, on="fid", how="left")
result_df = result_df.join(followers_count_df, on="fid", how="left")
result_df = result_df.fill_null(0)

/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_49275/1728189285.py:3: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  result_df = bots_df.join(following_count_df, on="fid", how="left")
/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_49275/1728189285.py:4: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  result_df = result_df.join(followers_count_df, on="fid", how="left")


In [23]:
result_df

created_at,updated_at,custody_address,registered_at,fid,verifications_count,recasts_count,likes_count,total_casts_count,reply_casts_count,casts_count,bot,text,following_count,followers_count
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,i64,i64,i64,i64,i64,i64,bool,str,i64,i64
2024-04-04 00:34:26.279484,2024-06-07 08:11:42.199,"b""\xcb\x1b\xbc\x1e\xbd;\xb6\x8dz\xe9\xf1\xc2\xf0\x1c\x0c\xc8\x8a\xf3\xd4\xef""",2024-04-04 00:34:11 UTC,446097,1,155,869,339,308,31,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic comments""",415,357
2023-09-05 23:07:28.414146,2024-06-07 16:44:01.621,"b""k\x0b\xda?/\xfe\xd5\xef\xc8?\xa8\xc0$\xac\xff\x1d\xd4W\x93\xf1""",2023-11-07 19:42:51 UTC,3,6,6280,57995,32634,24037,8597,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: farcaster, warpcast, notificat""",883,10448
2023-09-05 23:07:32.549415,2024-06-07 16:44:01.625,"b""\xc6\xe3\x00K\x0eT\xa9\x1d\xa8\xd8z\xce\x80\xb6\xab\xc6M#\xe3?""",2023-11-07 19:42:51 UTC,8,1,458,3459,3795,2666,1129,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: zora, mints, eclipse""",188,9079
2023-09-05 23:07:33.936715,2024-06-07 16:44:01.628,"b""\xb45u\xd5\x8cI\xd2\xbe\x88\x0e\x0f\x82\x1d\xdfQ\xd1\\x13\xc3v""",2023-11-07 19:42:51 UTC,12,2,3584,26667,7592,6468,1124,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: bountycaster, features, crypto""",328,8494
2023-09-05 23:07:28.009240,2024-06-07 16:44:01.629,"b""A\x14\xe3>\xb81\x85\x86I\xea7\x02\xe1\xc9\xa2\xdb?bdF""",2023-11-07 19:42:51 UTC,2,10,1535,14352,8069,6454,1615,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: Farcaster dev scaling""",465,8939
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-19 15:34:21.408148,2024-10-02 18:33:34.503,"b""\x99v\x94}\x8c\xc7\x1fMo\x16Cf\xbc\x86\x83\xdd1:\x84c""",2024-10-02 18:33:23 UTC,361320,4,2390,2683,2534,2416,118,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic, inane, spammy""",213,373
2024-03-20 21:02:12.707136,2024-10-03 15:33:11.612,"b""\xf0\x09\x8dcQ\xbb\xf2\xf9^\x87\x83mP\x9cg\x1f\x8e\xb7Y\xba""",2024-10-03 15:33:03 UTC,399306,7,26,827,404,121,283,false,"""Bot or Not? Someone checked : Not a bot. Casts: nouns,prop,solar light,community,thanks Want to check someone else? Install the cast action below.""",152,35
2023-10-11 21:35:30.283588,2024-10-03 15:33:11.627,"b""\x91\xfbW\x9b\x02.\x09\xac\x90\xa7\xa0\xa4jZ(\xb2;\xe7v\xa3""",2024-10-03 15:33:03 UTC,21348,2,57,2743,1564,1184,380,false,"""Bot or Not? Someone checked : Not a bot. Casts: music, dataviz, events, random thoughts Want to check someone else? Install the cast action below.""",299,175


In [24]:
# bot_or_not_df.collect().filter(pl.col('fid')==3)

In [25]:
# result_df.unique(subset=['fid'])

In [26]:
links_df #.unique(subset=['id'])

fid,target_fid,hash,timestamp,created_at,updated_at,deleted_at,type,display_timestamp,id
i64,i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64
2,23,"b""\xf6\xb3y\x03f\x0b\x0f\x15""R\xf32\xf1orQ\x16\xc1[\xcb""",2021-07-22 21:38:55,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,22
2,18,"b""\xb9\x18\xc8\xf8\xb1_\xf4SU\x9b\x0f\x18\xce\x99D\xc5T\x12\x8a\x7f""",2021-07-22 21:38:59,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,23
2,20,"b""v\xb8\xaa\x96\x1c\x94\x8d\xaf\xc0c\xbf\xb8N\x88\x82\x87\xa6Q^q""",2021-07-22 21:39:03,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,25
2,8,"b""`e\xed\xd6\xb7\x9b\xf4H\x9d\x8fU\xcb\x98\x07\xac\x15kD\x95|""",2021-07-22 21:39:04,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,26
2,12,"b""4{*K\x02l\x05\x08\xe3\x88\x84\xc5\xc2\xfb\xb6m\x9c=i\xdd""",2021-07-22 21:39:06,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,28
…,…,…,…,…,…,…,…,…,…
467312,389808,"b""\x07\xf9`&\xd6\x0foq\xce\x7f\x1d\xe7\xa9\x86B\x9f\xba#+t""",2024-09-12 12:08:20,2024-09-12 12:08:20.457983,2024-10-03 16:56:28.567,2024-10-03 16:56:29,"""follow""",null,11873708645
368382,387244,"b""\xef\xe0\x09\x8f\xb0\x02b9\x06\xab\x15\xa6\x8di\x81v\x9ezR\xbd""",2024-10-03 16:56:46,2024-09-17 16:04:19.132916,2024-10-03 16:56:46.044,null,"""follow""",null,11874838125
379818,294370,"b""\x94\x90\xfc\x96\xf0\xf6\xbe\x7fME-r\x9e\xc2\xd0\xcd\xf2J\x1bH""",2024-10-03 16:56:52,2024-10-03 16:56:52.402420,2024-10-03 16:56:52.402420,null,"""follow""",null,11878799175


In [27]:
result_df

created_at,updated_at,custody_address,registered_at,fid,verifications_count,recasts_count,likes_count,total_casts_count,reply_casts_count,casts_count,bot,text,following_count,followers_count
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,i64,i64,i64,i64,i64,i64,bool,str,i64,i64
2024-04-04 00:34:26.279484,2024-06-07 08:11:42.199,"b""\xcb\x1b\xbc\x1e\xbd;\xb6\x8dz\xe9\xf1\xc2\xf0\x1c\x0c\xc8\x8a\xf3\xd4\xef""",2024-04-04 00:34:11 UTC,446097,1,155,869,339,308,31,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic comments""",415,357
2023-09-05 23:07:28.414146,2024-06-07 16:44:01.621,"b""k\x0b\xda?/\xfe\xd5\xef\xc8?\xa8\xc0$\xac\xff\x1d\xd4W\x93\xf1""",2023-11-07 19:42:51 UTC,3,6,6280,57995,32634,24037,8597,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: farcaster, warpcast, notificat""",883,10448
2023-09-05 23:07:32.549415,2024-06-07 16:44:01.625,"b""\xc6\xe3\x00K\x0eT\xa9\x1d\xa8\xd8z\xce\x80\xb6\xab\xc6M#\xe3?""",2023-11-07 19:42:51 UTC,8,1,458,3459,3795,2666,1129,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: zora, mints, eclipse""",188,9079
2023-09-05 23:07:33.936715,2024-06-07 16:44:01.628,"b""\xb45u\xd5\x8cI\xd2\xbe\x88\x0e\x0f\x82\x1d\xdfQ\xd1\\x13\xc3v""",2023-11-07 19:42:51 UTC,12,2,3584,26667,7592,6468,1124,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: bountycaster, features, crypto""",328,8494
2023-09-05 23:07:28.009240,2024-06-07 16:44:01.629,"b""A\x14\xe3>\xb81\x85\x86I\xea7\x02\xe1\xc9\xa2\xdb?bdF""",2023-11-07 19:42:51 UTC,2,10,1535,14352,8069,6454,1615,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: Farcaster dev scaling""",465,8939
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-19 15:34:21.408148,2024-10-02 18:33:34.503,"b""\x99v\x94}\x8c\xc7\x1fMo\x16Cf\xbc\x86\x83\xdd1:\x84c""",2024-10-02 18:33:23 UTC,361320,4,2390,2683,2534,2416,118,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic, inane, spammy""",213,373
2024-03-20 21:02:12.707136,2024-10-03 15:33:11.612,"b""\xf0\x09\x8dcQ\xbb\xf2\xf9^\x87\x83mP\x9cg\x1f\x8e\xb7Y\xba""",2024-10-03 15:33:03 UTC,399306,7,26,827,404,121,283,false,"""Bot or Not? Someone checked : Not a bot. Casts: nouns,prop,solar light,community,thanks Want to check someone else? Install the cast action below.""",152,35
2023-10-11 21:35:30.283588,2024-10-03 15:33:11.627,"b""\x91\xfbW\x9b\x02.\x09\xac\x90\xa7\xa0\xa4jZ(\xb2;\xe7v\xa3""",2024-10-03 15:33:03 UTC,21348,2,57,2743,1564,1184,380,false,"""Bot or Not? Someone checked : Not a bot. Casts: music, dataviz, events, random thoughts Want to check someone else? Install the cast action below.""",299,175


In [28]:
fnames_df = pl.read_parquet(f"{DATA_PATH}/farcaster-fnames-0-1730134800.parquet")
# 320189

In [29]:
# links_df = links_between_bots_df

# # G = nx.DiGraph()

# nodes = []
# for row in links_df.iter_rows():
#     # nodes.append([row[0], row[1]])

# # plt.figure(figsize=(10, 10))
# # pos = nx.spring_layout(G, seed=42)  # Positioning of the nodes
# # nx.draw(G, pos, with_labels=True, node_size=500, node_color="lightblue", font_size=10, font_weight="bold", edge_color="gray")

# # # Optional: Save the plot as an image
# # # plt.savefig("bot_graph_connections.png", format="PNG")

# # # Show the plot
# # plt.show()

In [30]:
# links_between_bots_df

In [31]:
# for row in links_between_bots_df.iter_rows():
#     print(row)
#     break

In [33]:
# len(nodes)

In [106]:
counts = ['fid','verifications_count', 'recasts_count', 'likes_count', 'total_casts_count','reply_casts_count', 'casts_count']

In [115]:
# result_df.filter(pl.col('bot')==True).sort('likes_count').tail().select(counts) #.describe()
result_df.filter(pl.col('bot')==True).sample(seed=42)

created_at,updated_at,custody_address,registered_at,fid,verifications_count,recasts_count,likes_count,total_casts_count,reply_casts_count,casts_count,bot,text,following_count,followers_count
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,u32,u32,u32,u32,u32,u32,bool,str,u32,u32
2024-02-16 19:25:01.951446,2024-06-08 01:14:21.282,"b""(WN\xb0o\xcb\xf8\xde<\xb6\xec\xc7\\xd3\xa9\x10\xabE\xa6{""",2024-02-16 19:24:51 UTC,352594,5,1916,23069,4566,4442,124,true,"""Bot or Not? Someone checked : This user replies like a bot: generic responses""",391,300


In [119]:
fnames_df.filter(pl.col('fid')==1)

created_at,updated_at,custody_address,expires_at,fid,deleted_at,fname
datetime[ns],datetime[ns],binary,datetime[ns],i64,datetime[ns],str
2023-09-05 23:07:28.042699,2024-06-07 16:44:02.341,"b""\x87sD'@\xc1|\x9d\x0f\x0b\x87\x02,r/\x9a\x13b\x06\xed""",null,1,null,"""farcaster"""
2023-09-05 23:07:28.045141,2024-06-07 16:44:02.372,"b""\x86\x92L7\xa974\xe8a\x1e\xb0\x81#\x89(\xa9\xd1\x8ac\xc0""",null,1,null,"""warpcast.eth"""
2023-09-05 23:07:28.048640,2024-06-07 16:44:02.386,"b""\x86\x92L7\xa974\xe8a\x1e\xb0\x81#\x89(\xa9\xd1\x8ac\xc0""",null,1,null,"""farcaster.eth"""


In [112]:
profile_with_addresses_df = pl.read_parquet(f"{DATA_PATH}/farcaster-profile_with_addresses-0-1730134800.parquet")

In [118]:
profile_with_addresses_df.filter(pl.col('fid')==1)

fname,display_name,avatar_url,bio,verified_addresses,updated_at,fid
str,str,str,str,str,datetime[ns],i64
"""farcaster""","""Farcaster""","""https://i.imgur.com/I2rEbPF.png""","""A sufficiently decentralized social network. farcaster.xyz""","""[""0x86924c37a93734e8611eb081238928a9d18a63c0""]""",2024-06-07 16:44:02.646,1


In [34]:
links_df

fid,target_fid,hash,timestamp,created_at,updated_at,deleted_at,type,display_timestamp,id
i64,i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64
2,23,"b""\xf6\xb3y\x03f\x0b\x0f\x15""R\xf32\xf1orQ\x16\xc1[\xcb""",2021-07-22 21:38:55,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,22
2,18,"b""\xb9\x18\xc8\xf8\xb1_\xf4SU\x9b\x0f\x18\xce\x99D\xc5T\x12\x8a\x7f""",2021-07-22 21:38:59,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,23
2,20,"b""v\xb8\xaa\x96\x1c\x94\x8d\xaf\xc0c\xbf\xb8N\x88\x82\x87\xa6Q^q""",2021-07-22 21:39:03,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,25
2,8,"b""`e\xed\xd6\xb7\x9b\xf4H\x9d\x8fU\xcb\x98\x07\xac\x15kD\x95|""",2021-07-22 21:39:04,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,26
2,12,"b""4{*K\x02l\x05\x08\xe3\x88\x84\xc5\xc2\xfb\xb6m\x9c=i\xdd""",2021-07-22 21:39:06,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,28
…,…,…,…,…,…,…,…,…,…
467312,389808,"b""\x07\xf9`&\xd6\x0foq\xce\x7f\x1d\xe7\xa9\x86B\x9f\xba#+t""",2024-09-12 12:08:20,2024-09-12 12:08:20.457983,2024-10-03 16:56:28.567,2024-10-03 16:56:29,"""follow""",null,11873708645
368382,387244,"b""\xef\xe0\x09\x8f\xb0\x02b9\x06\xab\x15\xa6\x8di\x81v\x9ezR\xbd""",2024-10-03 16:56:46,2024-09-17 16:04:19.132916,2024-10-03 16:56:46.044,null,"""follow""",null,11874838125
379818,294370,"b""\x94\x90\xfc\x96\xf0\xf6\xbe\x7fME-r\x9e\xc2\xd0\xcd\xf2J\x1bH""",2024-10-03 16:56:52,2024-10-03 16:56:52.402420,2024-10-03 16:56:52.402420,null,"""follow""",null,11878799175


In [19]:
links_df.filter(pl.col('deleted_at').is_null()).select("type").count()

type
u32
5539137


In [18]:
links_df.filter(pl.col('deleted_at').is_null() & (pl.col('type') == "follow")).select('type').count()

type
u32
5539135


In [38]:
df_filtered = links_df.filter(pl.col("deleted_at").is_null())
df_filtered = df_filtered.filter(pl.col('fid') != pl.col('target_fid'))
df_filtered = df_filtered.filter(pl.col("type") == "follow")

mutual_links = df_filtered.join(
    df_filtered,
    left_on=["fid", "target_fid"],
    right_on=["target_fid", "fid"],
    suffix="_reverse"
)


bot_or_not_filtered = bot_or_not_df.collect().filter(
    (pl.col("fid").is_in(mutual_links.select('fid').to_series())) |
    (pl.col("fid").is_in(mutual_links.select('target_fid').to_series()))
)
# bot_or_not_filtered
bot_or_not_with_index = bot_or_not_filtered.with_columns(
    pl.arange(0, bot_or_not_filtered.height).alias("index")
)

mutual_links_with_fid_index = mutual_links.join(
    bot_or_not_with_index.select([pl.col('fid'), pl.col('index').alias('fid_index')]),
    on='fid',
    how='left'
)

mutual_links_with_index = mutual_links_with_fid_index.join(
    bot_or_not_with_index.select([pl.col('fid'), pl.col('index').alias('target_fid_index')]),
    left_on='target_fid',
    right_on='fid',
    how='left'
)


mutual_links_with_index = mutual_links_with_index.with_columns(
    (pl.col("fid_index").cast(pl.Utf8) + " " + pl.col("target_fid_index").cast(pl.Utf8)).alias("connection")
)

mutual_links_with_index

/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_49275/3859943308.py:22: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  mutual_links_with_fid_index = mutual_links.join(
/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_49275/3859943308.py:28: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  mutual_links_with_index = mutual_links_with_fid_index.join(


fid,target_fid,hash,timestamp,created_at,updated_at,deleted_at,type,display_timestamp,id,hash_reverse,timestamp_reverse,created_at_reverse,updated_at_reverse,deleted_at_reverse,type_reverse,display_timestamp_reverse,id_reverse,fid_index,target_fid_index,connection
i64,i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64,i64,i64,str
23,2,"b""V\xaf\x0c\xcb\xef\xea&\xd6S\x1c\xc1\x82\x87\x11\xf6""\x1fLS\x96""",2021-07-22 22:19:06,2023-09-05 23:07:34.989427,2023-09-05 23:07:34.989427,null,"""follow""",null,6348,"b""\xf6\xb3y\x03f\x0b\x0f\x15""R\xf32\xf1orQ\x16\xc1[\xcb""",2021-07-22 21:38:55,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,22,8,4,"""8 4"""
18,2,"b""\xa6\xd3\x08q\xdc\x09\x8b`\xef\xdb\x03\x98Z\xc8\xb3\x8c\x9bO$\xab""",2021-07-25 15:39:43,2023-09-05 23:07:34.741851,2023-09-05 23:07:34.741851,null,"""follow""",null,5689,"b""\xb9\x18\xc8\xf8\xb1_\xf4SU\x9b\x0f\x18\xce\x99D\xc5T\x12\x8a\x7f""",2021-07-22 21:38:59,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,23,6,4,"""6 4"""
20,2,"b""\x07\xf3\x06@\xcb\x19u\xe2\xb1\x11\xea\xce\x7f\xab\x04\x93\xce\x86\xa0\x11""",2022-04-27 02:00:49,2023-09-05 23:07:34.847303,2023-09-05 23:07:34.847303,null,"""follow""",null,6104,"b""v\xb8\xaa\x96\x1c\x94\x8d\xaf\xc0c\xbf\xb8N\x88\x82\x87\xa6Q^q""",2021-07-22 21:39:03,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,25,7,4,"""7 4"""
8,2,"b""\x87\xb4\xb0\xac7\xed`\xf8\xe5\x8f0\xdc\xda\xfe\xf3\x98\xcd\x9d\xce2""",2021-09-19 19:50:34,2023-09-05 23:07:33.799636,2023-09-05 23:07:33.799636,null,"""follow""",null,4093,"b""`e\xed\xd6\xb7\x9b\xf4H\x9d\x8fU\xcb\x98\x07\xac\x15kD\x95|""",2021-07-22 21:39:04,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,26,2,4,"""2 4"""
12,2,"b""\xe5-*)\xd3j\xaf[/\xcf\x96\x93\xf9\xca&\xcc\x8cW\x06\xac""",2021-08-18 05:15:08,2023-09-05 23:07:34.198928,2023-09-05 23:07:34.198928,null,"""follow""",null,4796,"b""4{*K\x02l\x05\x08\xe3\x88\x84\xc5\xc2\xfb\xb6m\x9c=i\xdd""",2021-07-22 21:39:06,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,28,3,4,"""3 4"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
500590,404798,"b""8&\x0c\xcb\x8f\x80\xf5A<\xe3\xc3p\x0f\xdff:E\x9dk\xde""",2024-04-29 17:33:02,2024-04-29 17:33:04.714338,2024-04-29 17:33:04.714338,null,"""follow""",null,8511803409,"b""eN9L\xfe+hF\xe6\xd7`fCSI\x9f.\x9e\x85\xfc""",2024-10-03 16:44:12,2024-10-03 16:44:13.079137,2024-10-03 16:44:13.079137,null,"""follow""",null,11878796069,10855,6551,"""10855 6551"""
294370,368382,"b""\x90\x9b\xbeE|\x7f\x1b\x13\xdb\xd5\x8a\x91\xd4pN\xee\x00\x87\xe7+""",2024-09-18 00:05:18,2024-09-18 00:05:18.624146,2024-09-18 00:05:18.624146,null,"""follow""",null,11874894051,"b""\xcbc/\x1d\xbb\xfaj\x87.X\xe3\xf2\xe1\xf1\x95\xb9\x9f\x97\x8a\xde""",2024-10-03 16:47:25,2024-09-05 23:03:14.922785,2024-10-03 16:47:25.483,null,"""follow""",null,11872176263,3424,5037,"""3424 5037"""
3095,400437,"b""\*\x91T\x94\xdb:p\x97z\xb2wXJ\x8d\xc7|\xc6\xc9\x9e""",2024-06-21 17:33:44,2024-06-21 17:33:49.801937,2024-06-21 17:33:49.801937,null,"""follow""",null,11842819884,"b""\x1d\xffFD\x82\x8aM*_\x16\\x94\xd4dew\xcf\x8c6E""",2024-10-03 16:51:01,2024-04-27 12:11:12.795757,2024-10-03 16:51:00.633,null,"""follow""",null,8244774968,348,6277,"""348 6277"""


In [39]:
# bot_or_not_with_index.filter(pl.col("fid")==2)

In [40]:
connections_list = mutual_links_with_index.select("connection").to_series().to_list()
connections_list

['8 4',
 '6 4',
 '7 4',
 '2 4',
 '3 4',
 '10 4',
 '5 4',
 '12 4',
 '11 4',
 '13 4',
 '15 4',
 '16 4',
 '17 4',
 '9 4',
 '21 4',
 '22 4',
 '23 4',
 '20 4',
 '19 4',
 '18 4',
 '24 4',
 '25 4',
 '27 4',
 '28 4',
 '30 4',
 '29 4',
 '31 4',
 '33 4',
 '34 4',
 '35 4',
 '36 4',
 '38 4',
 '39 4',
 '40 4',
 '41 4',
 '42 4',
 '44 4',
 '45 4',
 '46 4',
 '48 4',
 '50 4',
 '51 4',
 '54 4',
 '55 4',
 '57 4',
 '58 4',
 '59 4',
 '60 4',
 '61 4',
 '62 4',
 '63 4',
 '65 4',
 '67 4',
 '69 4',
 '70 4',
 '72 4',
 '73 4',
 '74 4',
 '75 4',
 '77 4',
 '78 4',
 '79 4',
 '81 4',
 '82 4',
 '84 4',
 '85 4',
 '89 4',
 '91 4',
 '92 4',
 '93 4',
 '94 4',
 '95 4',
 '97 4',
 '96 4',
 '99 4',
 '101 4',
 '102 4',
 '103 4',
 '106 4',
 '107 4',
 '108 4',
 '110 4',
 '111 4',
 '112 4',
 '115 4',
 '116 4',
 '117 4',
 '11445 4',
 '120 4',
 '121 4',
 '122 4',
 '123 4',
 '124 4',
 '125 4',
 '126 4',
 '129 4',
 '128 4',
 '130 4',
 '131 4',
 '133 4',
 '134 4',
 '136 4',
 '137 4',
 '139 4',
 '140 4',
 '142 4',
 '144 4',
 '146 4',


In [99]:
with open('graph.txt', 'w') as f:
    f.write("\n".join(connections_list))

In [100]:
# bot_or_not = bot_or_not_df.collect()

In [101]:
print(bot_or_not_with_index.filter(pl.col("bot")==True).shape)

(4604, 16)


In [102]:
print(bot_or_not_with_index.filter(pl.col("bot")==False).shape)

(7052, 16)


In [125]:
train_sybils = bot_or_not_with_index.filter(pl.col("bot")==True).sample(300, seed=40).select("index").to_series().to_list()
train_benigns = bot_or_not_with_index.filter(pl.col("bot")==False).sample(300, seed=40).select("index").to_series().to_list()

In [126]:
test_sybils = bot_or_not_with_index.filter((pl.col("bot")==True) & (~pl.col("index").is_in(train_sybils))).select("index").to_series().to_list()
test_benigns = bot_or_not_with_index.filter((pl.col("bot")==False) & (~pl.col("index").is_in(train_benigns))).select("index").to_series().to_list()

In [127]:
benign_ids_str = " ".join(map(str, test_benigns))
sybil_ids_str = " ".join(map(str, test_sybils))

with open('test.txt', 'w') as f:
    f.write(benign_ids_str + "\n")
    f.write(sybil_ids_str + "\n")

In [128]:
benign_ids_str = " ".join(map(str, train_benigns))
sybil_ids_str = " ".join(map(str, train_sybils))

with open('train.txt', 'w') as f:
    f.write(benign_ids_str + "\n")
    f.write(sybil_ids_str + "\n")

In [129]:
bot_or_not_with_index

created_at,updated_at,custody_address,registered_at,fid,verifications_count,following_count,followers_count,recasts_count,likes_count,total_casts_count,reply_casts_count,casts_count,bot,text,index
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,u32,u32,u32,u32,u32,u32,u32,u32,bool,str,i64
2024-04-04 00:34:26.279484,2024-06-07 08:11:42.199,"b""\xcb\x1b\xbc\x1e\xbd;\xb6\x8dz\xe9\xf1\xc2\xf0\x1c\x0c\xc8\x8a\xf3\xd4\xef""",2024-04-04 00:34:11 UTC,446097,1,1024,902,155,869,339,308,31,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic comments""",0
2023-09-05 23:07:28.414146,2024-06-07 16:44:01.621,"b""k\x0b\xda?/\xfe\xd5\xef\xc8?\xa8\xc0$\xac\xff\x1d\xd4W\x93\xf1""",2023-11-07 19:42:51 UTC,3,6,3425,484305,6280,57995,32634,24037,8597,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: farcaster, warpcast, notificat""",1
2023-09-05 23:07:32.549415,2024-06-07 16:44:01.625,"b""\xc6\xe3\x00K\x0eT\xa9\x1d\xa8\xd8z\xce\x80\xb6\xab\xc6M#\xe3?""",2023-11-07 19:42:51 UTC,8,1,355,426833,458,3459,3795,2666,1129,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: zora, mints, eclipse""",2
2023-09-05 23:07:33.936715,2024-06-07 16:44:01.628,"b""\xb45u\xd5\x8cI\xd2\xbe\x88\x0e\x0f\x82\x1d\xdfQ\xd1\\x13\xc3v""",2023-11-07 19:42:51 UTC,12,2,670,387753,3584,26667,7592,6468,1124,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: bountycaster, features, crypto""",3
2023-09-05 23:07:28.009240,2024-06-07 16:44:01.629,"b""A\x14\xe3>\xb81\x85\x86I\xea7\x02\xe1\xc9\xa2\xdb?bdF""",2023-11-07 19:42:51 UTC,2,10,1320,405256,1535,14352,8069,6454,1615,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: Farcaster dev scaling""",4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-19 15:34:21.408148,2024-10-02 18:33:34.503,"b""\x99v\x94}\x8c\xc7\x1fMo\x16Cf\xbc\x86\x83\xdd1:\x84c""",2024-10-02 18:33:23 UTC,361320,4,594,865,2390,2683,2534,2416,118,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic, inane, spammy""",11651
2024-03-20 21:02:12.707136,2024-10-03 15:33:11.612,"b""\xf0\x09\x8dcQ\xbb\xf2\xf9^\x87\x83mP\x9cg\x1f\x8e\xb7Y\xba""",2024-10-03 15:33:03 UTC,399306,7,273,161,26,827,404,121,283,false,"""Bot or Not? Someone checked : Not a bot. Casts: nouns,prop,solar light,community,thanks Want to check someone else? Install the cast action below.""",11652
2023-10-11 21:35:30.283588,2024-10-03 15:33:11.627,"b""\x91\xfbW\x9b\x02.\x09\xac\x90\xa7\xa0\xa4jZ(\xb2;\xe7v\xa3""",2024-10-03 15:33:03 UTC,21348,2,553,907,57,2743,1564,1184,380,false,"""Bot or Not? Someone checked : Not a bot. Casts: music, dataviz, events, random thoughts Want to check someone else? Install the cast action below.""",11653


In [130]:
bot_or_not_with_index.filter(pl.col("bot")==False)
4_655 + 7_192

11847

In [131]:
bot_or_not_with_index

created_at,updated_at,custody_address,registered_at,fid,verifications_count,following_count,followers_count,recasts_count,likes_count,total_casts_count,reply_casts_count,casts_count,bot,text,index
datetime[ns],datetime[ns],binary,"datetime[ns, UTC]",i64,u32,u32,u32,u32,u32,u32,u32,u32,bool,str,i64
2024-04-04 00:34:26.279484,2024-06-07 08:11:42.199,"b""\xcb\x1b\xbc\x1e\xbd;\xb6\x8dz\xe9\xf1\xc2\xf0\x1c\x0c\xc8\x8a\xf3\xd4\xef""",2024-04-04 00:34:11 UTC,446097,1,1024,902,155,869,339,308,31,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic comments""",0
2023-09-05 23:07:28.414146,2024-06-07 16:44:01.621,"b""k\x0b\xda?/\xfe\xd5\xef\xc8?\xa8\xc0$\xac\xff\x1d\xd4W\x93\xf1""",2023-11-07 19:42:51 UTC,3,6,3425,484305,6280,57995,32634,24037,8597,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: farcaster, warpcast, notificat""",1
2023-09-05 23:07:32.549415,2024-06-07 16:44:01.625,"b""\xc6\xe3\x00K\x0eT\xa9\x1d\xa8\xd8z\xce\x80\xb6\xab\xc6M#\xe3?""",2023-11-07 19:42:51 UTC,8,1,355,426833,458,3459,3795,2666,1129,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: zora, mints, eclipse""",2
2023-09-05 23:07:33.936715,2024-06-07 16:44:01.628,"b""\xb45u\xd5\x8cI\xd2\xbe\x88\x0e\x0f\x82\x1d\xdfQ\xd1\\x13\xc3v""",2023-11-07 19:42:51 UTC,12,2,670,387753,3584,26667,7592,6468,1124,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: bountycaster, features, crypto""",3
2023-09-05 23:07:28.009240,2024-06-07 16:44:01.629,"b""A\x14\xe3>\xb81\x85\x86I\xea7\x02\xe1\xc9\xa2\xdb?bdF""",2023-11-07 19:42:51 UTC,2,10,1320,405256,1535,14352,8069,6454,1615,false,"""Bot or Not? Someone checked : Not a bot. Buildooor: Farcaster dev scaling""",4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-19 15:34:21.408148,2024-10-02 18:33:34.503,"b""\x99v\x94}\x8c\xc7\x1fMo\x16Cf\xbc\x86\x83\xdd1:\x84c""",2024-10-02 18:33:23 UTC,361320,4,594,865,2390,2683,2534,2416,118,false,"""Bot or Not? Someone checked : Not a bot. Reply guy/gal: generic, inane, spammy""",11651
2024-03-20 21:02:12.707136,2024-10-03 15:33:11.612,"b""\xf0\x09\x8dcQ\xbb\xf2\xf9^\x87\x83mP\x9cg\x1f\x8e\xb7Y\xba""",2024-10-03 15:33:03 UTC,399306,7,273,161,26,827,404,121,283,false,"""Bot or Not? Someone checked : Not a bot. Casts: nouns,prop,solar light,community,thanks Want to check someone else? Install the cast action below.""",11652
2023-10-11 21:35:30.283588,2024-10-03 15:33:11.627,"b""\x91\xfbW\x9b\x02.\x09\xac\x90\xa7\xa0\xa4jZ(\xb2;\xe7v\xa3""",2024-10-03 15:33:03 UTC,21348,2,553,907,57,2743,1564,1184,380,false,"""Bot or Not? Someone checked : Not a bot. Casts: music, dataviz, events, random thoughts Want to check someone else? Install the cast action below.""",11653


In [123]:
# print(bot_or_not_with_index.filter((pl.col("bot")==True) & (~pl.col("index").is_in(train_sybils))))
# print(bot_or_not_with_index.filter((pl.col("bot")==False) & (~pl.col("index").is_in(train_benigns))))

4_155 + 6_692 #+ 1000

10847

In [45]:
mutual_links_with_index #.filter(pl.col('fid')==pl.col('target_fid'))

fid,target_fid,hash,timestamp,created_at,updated_at,deleted_at,type,display_timestamp,id,hash_reverse,timestamp_reverse,created_at_reverse,updated_at_reverse,deleted_at_reverse,type_reverse,display_timestamp_reverse,id_reverse,fid_index,target_fid_index,connection
i64,i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64,binary,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,datetime[ns],i64,i64,i64,str
23,2,"b""V\xaf\x0c\xcb\xef\xea&\xd6S\x1c\xc1\x82\x87\x11\xf6""\x1fLS\x96""",2021-07-22 22:19:06,2023-09-05 23:07:34.989427,2023-09-05 23:07:34.989427,null,"""follow""",null,6348,"b""\xf6\xb3y\x03f\x0b\x0f\x15""R\xf32\xf1orQ\x16\xc1[\xcb""",2021-07-22 21:38:55,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,22,8,4,"""8 4"""
18,2,"b""\xa6\xd3\x08q\xdc\x09\x8b`\xef\xdb\x03\x98Z\xc8\xb3\x8c\x9bO$\xab""",2021-07-25 15:39:43,2023-09-05 23:07:34.741851,2023-09-05 23:07:34.741851,null,"""follow""",null,5689,"b""\xb9\x18\xc8\xf8\xb1_\xf4SU\x9b\x0f\x18\xce\x99D\xc5T\x12\x8a\x7f""",2021-07-22 21:38:59,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,23,6,4,"""6 4"""
20,2,"b""\x07\xf3\x06@\xcb\x19u\xe2\xb1\x11\xea\xce\x7f\xab\x04\x93\xce\x86\xa0\x11""",2022-04-27 02:00:49,2023-09-05 23:07:34.847303,2023-09-05 23:07:34.847303,null,"""follow""",null,6104,"b""v\xb8\xaa\x96\x1c\x94\x8d\xaf\xc0c\xbf\xb8N\x88\x82\x87\xa6Q^q""",2021-07-22 21:39:03,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,25,7,4,"""7 4"""
8,2,"b""\x87\xb4\xb0\xac7\xed`\xf8\xe5\x8f0\xdc\xda\xfe\xf3\x98\xcd\x9d\xce2""",2021-09-19 19:50:34,2023-09-05 23:07:33.799636,2023-09-05 23:07:33.799636,null,"""follow""",null,4093,"b""`e\xed\xd6\xb7\x9b\xf4H\x9d\x8fU\xcb\x98\x07\xac\x15kD\x95|""",2021-07-22 21:39:04,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,26,2,4,"""2 4"""
12,2,"b""\xe5-*)\xd3j\xaf[/\xcf\x96\x93\xf9\xca&\xcc\x8cW\x06\xac""",2021-08-18 05:15:08,2023-09-05 23:07:34.198928,2023-09-05 23:07:34.198928,null,"""follow""",null,4796,"b""4{*K\x02l\x05\x08\xe3\x88\x84\xc5\xc2\xfb\xb6m\x9c=i\xdd""",2021-07-22 21:39:06,2023-09-05 23:07:30.968478,2023-09-05 23:07:30.968478,null,"""follow""",null,28,3,4,"""3 4"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
500590,404798,"b""8&\x0c\xcb\x8f\x80\xf5A<\xe3\xc3p\x0f\xdff:E\x9dk\xde""",2024-04-29 17:33:02,2024-04-29 17:33:04.714338,2024-04-29 17:33:04.714338,null,"""follow""",null,8511803409,"b""eN9L\xfe+hF\xe6\xd7`fCSI\x9f.\x9e\x85\xfc""",2024-10-03 16:44:12,2024-10-03 16:44:13.079137,2024-10-03 16:44:13.079137,null,"""follow""",null,11878796069,10855,6551,"""10855 6551"""
294370,368382,"b""\x90\x9b\xbeE|\x7f\x1b\x13\xdb\xd5\x8a\x91\xd4pN\xee\x00\x87\xe7+""",2024-09-18 00:05:18,2024-09-18 00:05:18.624146,2024-09-18 00:05:18.624146,null,"""follow""",null,11874894051,"b""\xcbc/\x1d\xbb\xfaj\x87.X\xe3\xf2\xe1\xf1\x95\xb9\x9f\x97\x8a\xde""",2024-10-03 16:47:25,2024-09-05 23:03:14.922785,2024-10-03 16:47:25.483,null,"""follow""",null,11872176263,3424,5037,"""3424 5037"""
3095,400437,"b""\*\x91T\x94\xdb:p\x97z\xb2wXJ\x8d\xc7|\xc6\xc9\x9e""",2024-06-21 17:33:44,2024-06-21 17:33:49.801937,2024-06-21 17:33:49.801937,null,"""follow""",null,11842819884,"b""\x1d\xffFD\x82\x8aM*_\x16\\x94\xd4dew\xcf\x8c6E""",2024-10-03 16:51:01,2024-04-27 12:11:12.795757,2024-10-03 16:51:00.633,null,"""follow""",null,8244774968,348,6277,"""348 6277"""


In [50]:
df1 = mutual_links_with_index.join(bot_or_not_filtered.select(["fid", "bot"]), how="left", on="fid")
df1.select(["fid_index","target_fid_index","bot"]).write_parquet(f"{DATA_PATH}/test-bots.parquet")

/var/folders/v9/zdrfs19x765_sy8y01lgc13m0000gn/T/ipykernel_49275/2636839249.py:1: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df1 = mutual_links_with_index.join(bot_or_not_filtered.select(["fid", "bot"]), how="left", on="fid")
